In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
from tqdm import tqdm
import os
import data_utils
import model_utils
from attack_utils import get_CSMIA_case_by_case_results, CSMIA_attack, LOMIA_attack
from data_utils import oneHotCatVars, filter_random_data_by_conf_score
from vulnerability_score_utils import get_vulnerability_score, draw_hist_plot
from experiment_utils import MIAExperiment
from disparity_inference_utils import get_confidence_array, draw_confidence_array_scatter, get_indices_by_group_condition, get_corr_btn_sens_and_out_per_subgroup, get_slopes, get_angular_difference, calculate_stds, get_mutual_info_btn_sens_and_out_per_subgroup
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.neural_network._base import ACTIVATIONS
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import roc_curve, auc, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA
from sklearn.inspection import permutation_importance
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate
import pickle
# import utils
import copy

import matplotlib as mpl

# Setting the font family, size, and weight globally
mpl.rcParams['font.family'] = 'DejaVu Sans'
mpl.rcParams['font.size'] = 8
mpl.rcParams['font.weight'] = 'light'

In [5]:
i = -0.4
j = -0.1
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'correlation': 0,
            'subgroup_col_name': 'SEX',
            'marginal_prior': 1,
            'corr_btn_sens_and_output_per_subgroup': (i, j),
            # 'fixed_corr_in_test_data': True
    }, shortname = f"Corr_btn_sens_and_output_for_male_({i})_for_female_({j})"
)

[0, 1]
{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}}


100%|██████████| 2/2 [00:01<00:00,  1.56it/s]


{0: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}, 1: {(0, 1): 7500, (0, 0): 5000, (1, 1): 5000, (1, 0): 7500}}


100%|██████████| 2/2 [00:01<00:00,  1.62it/s]


In [4]:
i = -0.4
j = -0.1
experiment = MIAExperiment(sampling_condition_dict_list = 
    {
            'correlation': 0,
            'subgroup_col_name': 'SEX_CODE',
            'marginal_prior': 1,
            'corr_btn_sens_and_output_per_subgroup': (i, j),
            # 'fixed_corr_in_test_data': True
    }, shortname = f"Corr_btn_sens_and_output_for_male_({i})_for_female_({j})", name='Texas100', sensitive_column='ETHNICITY'
)

[0, 1]
{0: {(0, 1): 8750, (0, 0): 3750, (1, 1): 3750, (1, 0): 8750}, 1: {(0, 1): 6875, (0, 0): 5625, (1, 1): 5625, (1, 0): 6875}}


100%|██████████| 2/2 [00:00<00:00,  4.52it/s]


{0: {(0, 1): 6250, (0, 0): 6250, (1, 1): 6250, (1, 0): 6250}, 1: {(0, 1): 5625, (0, 0): 6875, (1, 1): 6875, (1, 0): 5625}}


100%|██████████| 2/2 [00:00<00:00,  5.39it/s]


In [5]:
save_model=True
print(f"Training classifier for experiment: {experiment}")
try:
    experiment.clf = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')
    print(f"Loaded classifier for experiment from file: {experiment}")
except:
    # clf = model_utils.get_model(max_iter=500, hidden_layer_sizes=(256, 256))
    base_model = model_utils.get_model(max_iter=500)
    experiment.clf = copy.deepcopy(base_model)
    experiment.clf.fit(experiment.X_test, experiment.y_te_onehot)

    if save_model:
        model_utils.save_model(experiment.clf, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_target_model_only_on_test_dummy.pkl')

Training classifier for experiment: Texas100_correlation_0_subgroup_col_name_SEX_CODE_marginal_prior_1_corr_btn_sens_and_output_per_subgroup_LPAREN-0.4, -0.1RPAREN
Loaded classifier for experiment from file: Texas100_correlation_0_subgroup_col_name_SEX_CODE_marginal_prior_1_corr_btn_sens_and_output_per_subgroup_LPAREN-0.4, -0.1RPAREN


In [6]:
from sklearn.neural_network import MLPClassifier

class MLPClassifierFC(MLPClassifier):
    def fit(self, X, y, sample_weight=None):
        """
            Fit the model to the given data.
        """
        if sample_weight is not None:
            # resample data according to sample weights
            n_samples = X.shape[0]
            sample_weight = np.asarray(sample_weight)/np.sum(sample_weight)
            sample_idxs = np.random.choice(n_samples, n_samples, p=sample_weight)
            X = X.iloc[sample_idxs]
            y = y[sample_idxs]
            
        return super().fit(X, y)

clf2 = MLPClassifierFC(max_iter=500)
clf2.coefs_ = experiment.clf.coefs_
clf2.intercepts_ = experiment.clf.intercepts_

In [10]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds, ErrorRate

subgroup_col_name = 'SEX_CODE'

In [8]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity, EqualizedOdds, ErrorRate

subgroup_col_name = 'SEX_CODE'

try:
    mitigator = model_utils.load_model(f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_mitigator_eqodds_test_{subgroup_col_name}.pkl')
except:
    constraint = EqualizedOdds()
    mitigator = ExponentiatedGradient(clf2, constraint)

    mitigator.fit(experiment.X_test, experiment.y_te, sensitive_features=experiment.X_test[f'{subgroup_col_name}_0'])

    model_utils.save_model(mitigator, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_mitigator_eqodds_test_{subgroup_col_name}.pkl')

In [14]:
mitigators = {}
mitigators['db_none'] = mitigator

In [11]:
clf3 = MLPClassifierFC(max_iter=500)
clf3.coefs_ = experiment.clf.coefs_
clf3.intercepts_ = experiment.clf.intercepts_
mitigator_db_p01 = ExponentiatedGradient(clf3, EqualizedOdds(difference_bound=0.01))
mitigator_db_p01.fit(experiment.X_test, experiment.y_te, sensitive_features=experiment.X_test[f'{subgroup_col_name}_0'])

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.EqualizedOdds object at 0x16fe43880>,
                      estimator=MLPClassifierFC(max_iter=500),
                      nu=0.0008769727952597349)

In [21]:
model_utils.save_model(mitigator_db_p01, f'<PATH_TO_MODEL>/{experiment.ds.ds.filenameroot}_mitigator_eqodds_db_p01_test_{subgroup_col_name}.pkl')

In [12]:
y_te_pred = mitigator_db_p01.predict(experiment.X_test)
subgroup_vals_te = experiment.X_test[f'SEX_CODE_0'].to_numpy().ravel()
equalized_odds_difference(experiment.y_te.ravel(), y_te_pred, sensitive_features=subgroup_vals_te)

0.02176

In [14]:
y_te_pred = mitigator_db_p01.predict(experiment.X_test)
subgroup_vals_te = experiment.X_test[f'SEX_CODE_0'].to_numpy().ravel()
demographic_parity_difference(experiment.y_te.ravel(), y_te_pred, sensitive_features=subgroup_vals_te)

0.0020399999999999308

In [15]:
subgroup_vals_te = experiment.X_test[f'{subgroup_col_name}_1'].to_numpy().ravel()
sens_pred_LOMIA = LOMIA_attack(experiment, mitigator_db_p01, experiment.X_test, experiment.y_te, experiment.ds.ds.meta)

In [16]:
correct_indices_LOMIA = (sens_pred_LOMIA == experiment.X_test[[f'{experiment.ds.ds.meta["sensitive_column"]}_1']].to_numpy().ravel())

In [17]:
print(100 * (correct_indices_LOMIA[np.where(subgroup_vals_te==0)[0]].mean()-correct_indices_LOMIA[np.where(subgroup_vals_te==1)[0]].mean()))

13.280000000000003


In [18]:
from model_utils import predict_proba_for_mitiagtor
subgroup_vals_tr = experiment.X_train[f'{subgroup_col_name}_1'].to_numpy().ravel()
male_indices = np.where(subgroup_vals_tr==0)[0]
female_indices = np.where(subgroup_vals_tr==1)[0]
y_tr_pred = np.argmax(mitigator_db_p01._pmf_predict(experiment.X_train), axis=1)

In [19]:
100 * accuracy_score(experiment.y_tr.ravel()[male_indices], y_tr_pred[male_indices])

71.788

In [20]:
100 * accuracy_score(experiment.y_tr.ravel()[female_indices], y_tr_pred[male_indices])

68.108

In [18]:
clf4 = MLPClassifierFC(max_iter=500)
clf4.coefs_ = experiment.clf.coefs_
clf4.intercepts_ = experiment.clf.intercepts_
mitigator_dp_db_p01 = ExponentiatedGradient(clf4, DemographicParity(difference_bound=0.01))
mitigator_dp_db_p01.fit(experiment.X_test, experiment.y_te, sensitive_features=experiment.X_test[f'{subgroup_col_name}_0'])

ExponentiatedGradient(constraints=<fairlearn.reductions._moments.utility_parity.DemographicParity object at 0x32458b8b0>,
                      estimator=MLPClassifierFC(max_iter=500),
                      nu=0.0008548056047151522)

In [16]:
get_CSMIA_case_by_case_results(mitigator_db_p01, experiment.X_test, experiment.y_te, experiment.ds, subgroup_col_name='SEX', sensitive_col_name='MAR_1')

,1,0,Overall
Case 1,4145 (65.121),5573 (71.0687),68.5056
Case 2,17798 (55.1002),16661 (66.8666),60.8624
Case 3,3057 (56.3745),2766 (56.0854),56.2350
Case All Cases,25000 (56.8906),25000 (66.5318),61.7476


In [27]:
y_tr_pred = mitigator_db_p01.predict(experiment.X_train)
subgroup_vals_tr = experiment.X_train[f'SEX_0'].to_numpy().ravel()
equalized_odds_difference(experiment.y_tr.ravel(), y_tr_pred, sensitive_features=subgroup_vals_tr)

0.06296000000000002

In [28]:
y_te_pred = mitigator_db_p01.predict(experiment.X_test)
subgroup_vals_te = experiment.X_test[f'SEX_0'].to_numpy().ravel()
equalized_odds_difference(experiment.y_te.ravel(), y_te_pred, sensitive_features=subgroup_vals_te)

0.017199999999999993

In [21]:
y_tr_pred.shape

(50000,)

In [12]:
get_CSMIA_case_by_case_results(mitigator, experiment.X_test, experiment.y_te, experiment.ds, subgroup_col_name='SEX', sensitive_col_name='MAR_1')

,1,0,Overall
Case 1,3750 (65.9039),5272 (69.5954),68.1096
Case 2,18077 (52.4326),16885 (67.0515),59.3924
Case 3,3173 (57.1429),2843 (58.2051),57.6786
Case All Cases,25000 (55.0063),25000 (66.4534),60.7494


In [13]:
get_CSMIA_case_by_case_results(experiment.clf, experiment.X_test, experiment.y_te, experiment.ds, subgroup_col_name='SEX', sensitive_col_name='MAR_1')

,1,0,Overall
Case 1,1998 (63.3056),4075 (63.8072),63.6483
Case 2,19357 (55.7876),17947 (74.5631),65.0548
Case 3,3645 (58.75),2978 (55.9896),57.3698
Case All Cases,25000 (56.9703),25000 (69.6637),63.5705
